# Digital music shop data analysis

In this project, I worked on data analysis of a database of a fictitious degital music shop like iTunes store. This database is called Chinook and publicly available [from here](https://github.com/lerocha/chinook-database).
I will work with SQLite portable database 'chinook.db'. The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. 

#### Chinook database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
--All tables and views in chinook database
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql
SELECT * FROM album LIMIT 5;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [5]:
%%sql
SELECT * FROM track LIMIT 5;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


## Which albums should we purchase for the store?

**Situation:** We want to introduce 3 new albums into our store and have 4 candidates which artists and genres are as follows.

|  Artist Name          |  Genre   |
| --------------------- | -------- |
|  Regal                |  Hip-Hop |
|  Red Tone             |  Punk    |
|  Meteor and the Girls |  Pop     |
|  Slim Jim Bite        |  Blues   |

Which albums should we choose?

In [6]:
%%sql
/*
Genres and the number of the corresponding tracks sold in USA
Join 4 tables genre, track, invoice_line, invoice where billing_country is USA
Group by genre name
*/

WITH genre_tracks
    AS (
        SELECT 
            genre.name,
            COUNT(*) AS n_track_sold
          FROM genre
          LEFT JOIN track
            ON genre.genre_id = track.genre_id
          LEFT JOIN invoice_line AS il
            ON track.track_id = il.track_id
          LEFT JOIN invoice
            ON il.invoice_id = invoice.invoice_id
         WHERE invoice.billing_country = 'USA'
         GROUP BY genre.name
     )
    
--Percentage of each genre in all genres in terms of the number of tracks
SELECT
    *,
    ROUND((n_track_sold/(
                  SELECT TOTAL(n_track_sold) 
                  FROM genre_tracks
                 ))*100, 1) AS percentage    
  FROM genre_tracks
 ORDER BY percentage DESC;


Done.


name,n_track_sold,percentage
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Latin,22,2.1
Pop,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


According to this data, we can say we should purchase Red Tone, Meteor and the Girls, and Slim Jim Bites for the store.

## Who is the best sales support agent?

In [7]:
%%sql
SELECT DISTINCT title AS all_unique_titles FROM employee

Done.


all_unique_titles
General Manager
Sales Manager
Sales Support Agent
IT Manager
IT Staff


Each cutomer is assigned to a sales support agent at their first purchase. Who is performing better or worse in terms of sales?

In [8]:
%%sql
/*
Sales support agents and their total sales.
Join 3 tables employee, customer, invoice.
Sum of total in invoice group by employee name.
*/
SELECT 
    e.first_name || ' ' || e.last_name AS employee_name,
    e.reports_to,
    e.birthdate,
    e.hire_date,
    e.country,
    SUM(i.total) AS total_sales
  FROM employee AS e
  LEFT JOIN customer AS c
    ON e.employee_id = c.support_rep_id
 INNER JOIN invoice AS i
    ON c.customer_id = i.customer_id
 WHERE e.title = 'Sales Support Agent'
 GROUP BY employee_name;

Done.


employee_name,reports_to,birthdate,hire_date,country,total_sales
Jane Peacock,2,1973-08-29 00:00:00,2017-04-01 00:00:00,Canada,1731.510000000004
Margaret Park,2,1947-09-19 00:00:00,2017-05-03 00:00:00,Canada,1584.0000000000032
Steve Johnson,2,1965-03-03 00:00:00,2017-10-17 00:00:00,Canada,1393.9200000000028


In [9]:
%%sql
SELECT DISTINCT country FROM employee

Done.


country
Canada


Only thing that can be seen here is the longer employees have been working in the store, the more sales they have. That is just trivial.

## Sales by country

In [10]:
%%sql
/*
For each country,
1. total number of customers
2. total value of sales
3. avg. value of sales per customers
4. avg. order value
Countries with only one customer are grouped into "Other".
*/

--Give new label 1 if the country is in "Other"
WITH country_other 
    AS (
        SELECT
            country,
            COUNT(DISTINCT customer_id) AS n_customers,
            CASE
                WHEN COUNT(DISTINCT customer_id) = 1 
                THEN 1
                ELSE 0
             END AS other_or_not
         FROM customer
        GROUP BY country
      )

--Group by country, bring "Other" at the bottom
SELECT
    c2.country,
    COUNT(DISTINCT c2.customer_id) AS n_customers,
    SUM(i2.total) AS total_sales,
    AVG(c2.sales_per_customer) AS avg_sales_p_customer,
    AVG(i2.total) AS avg_order_value
  FROM ( 
        --Relabel country column, get sales per customer
        SELECT
            c.customer_id,
            CASE
                WHEN co.other_or_not = 1
                THEN "Other"
                ELSE c.country
             END AS country,
            co.other_or_not,
            SUM(i1.total) AS sales_per_customer
          FROM customer AS c
         INNER JOIN country_other AS co
            ON c.country = co.country
         INNER JOIN invoice AS i1
            ON c.customer_id = i1.customer_id
         GROUP BY i1.customer_id
      ) AS c2
 INNER JOIN invoice AS i2
    ON c2.customer_id = i2.customer_id
 GROUP BY c2.country
 ORDER BY c2.other_or_not

Done.


country,n_customers,total_sales,avg_sales_p_customer,avg_order_value
Brazil,5,427.6799999999999,87.55819672131153,7.011147540983605
Canada,8,535.59,69.62565789473678,7.047236842105264
Czech Republic,2,273.23999999999995,138.20399999999992,9.107999999999999
France,5,389.07000000000005,78.42779999999999,7.781400000000001
Germany,4,334.61999999999995,83.90853658536594,8.161463414634145
India,2,183.14999999999995,96.40714285714282,8.72142857142857
Portugal,2,185.13,91.48965517241383,6.3837931034482756
USA,13,1040.4900000000005,81.30091603053418,7.942671755725194
United Kingdom,3,245.52,83.47821428571426,8.768571428571429
Other,15,1094.94,76.12897959183678,7.448571428571429


According to the average order value, 
1. Czech Republic
2. United Kingdom
3. India

might have potential for further growth.
However the number of samples is too small to make accurate prediction.

## Ratio of album purchases vs individual purchases

Customers have 2 types of purchase options:
1. Purchase by albums
2. Purchase by individual tracks

Customers can't mix 2 types of purchases.

**Situation**: The store is considering to change their purchasing strategy such that they purchase only popular tracks and not entire albums to save money. What would its impact on revenue be like?

I would like to figure out the percentage of each type of purchases.

#### Bad example

I first tried counting number of albums each invoice consists of and regarding all purchases containing only a single album as album purchases. However this overestimates album purchases since some customers purchase only a few tracks from the same album.

In [13]:
%%sql
--Bad example!
SELECT
    SUM(
        CASE
            WHEN n_albums = 1
            THEN 1
            ELSE 0
         END
      ) AS album_purchase,
    SUM(
        CASE
            WHEN n_albums = 1
            THEN 1
            ELSE 0
         END
      )/CAST(COUNT(*) AS FLOAT) AS percentage
  FROM (
        SELECT
            il.invoice_id,
            COUNT(DISTINCT t.album_id) AS n_albums
          FROM invoice_line AS il
         INNER JOIN track AS t
            ON il.track_id = t.track_id
         GROUP BY il.invoice_id
      )


Done.


album_purchase,percentage
171,0.2785016286644951


### Solutions

I found 3 different queries to achieve the table of the purpose.

#### Solution 1

In [30]:
%%sql
/*
For each type of purchase (album or indivisual tracks)
1. Number of invoices
2. Percentage in all purchases
*/

WITH
    --First track of each invoice. Used later as reference to album.
    invoice_first_track
        AS (
            SELECT
                il.invoice_id,
                MIN(il.track_id) AS first_track_id
              FROM invoice_line AS il
             GROUP BY il.invoice_id
          ),
    --Invoices and their tracks
    invoice_tracks
        AS (
            SELECT
                il.invoice_id,
                il.track_id
              FROM invoice_line AS il
          ),
    --Assign each invoice the tracks of the album referred to by the first With clause
    album_tracks
        AS (
            SELECT
                ift.invoice_id,
                t2.track_id
              FROM invoice_first_track AS ift
             INNER JOIN track AS t1
                ON ift.first_track_id = t1.track_id
             INNER JOIN track AS t2
                ON t1.album_id = t2.album_id
          ),
    --invoice_id of differences of invoice tracks and referred album tracks
    differences
        AS (
            SELECT DISTINCT invoice_id
              FROM (
                    SELECT *
                      FROM invoice_tracks
                    EXCEPT
                    SELECT *
                      FROM album_tracks
                   )
             UNION
            SELECT DISTINCT invoice_id
              FROM (
                    SELECT *
                      FROM album_tracks
                    EXCEPT
                    SELECT *
                      FROM invoice_tracks
                   )
          )

/*
In the subquery at FROM clause, label purchase type of each invoice 
by whether invoice_id is found in "differences" table.
Then group by this label and count invoice_id.
*/
SELECT
    purchase_type,
    COUNT(invoice_id) AS number_of_invoices,
    (
     COUNT(invoice_id)/CAST((SELECT COUNT(*) FROM invoice) AS Float)
    )*100 AS percentage
  FROM (
        SELECT
            invoice_id,
          CASE
              WHEN invoice_id IN differences
              THEN 'individual'
              ELSE 'album'
          END AS purchase_type
         FROM invoice
        ORDER BY invoice_id
       )
 GROUP BY purchase_type;

Done.


purchase_type,number_of_invoices,percentage
album,114,18.566775244299674
individual,500,81.43322475570032


#### Solution 2

In [32]:
%%sql
/*
For each type of purchase (album or indivisual tracks)
1. Number of invoices
2. Percentage in all purchases
*/

WITH
    --First track of each invoice. Used later as reference to album.
    invoice_first_track
        AS (
            SELECT
                il.invoice_id,
                MIN(il.track_id) AS first_track_id
              FROM invoice_line AS il
             GROUP BY il.invoice_id
          ),
    --Invoices and their tracks
    invoice_tracks
        AS (
            SELECT
                il.invoice_id,
                il.track_id
              FROM invoice_line AS il
          ),
    --Assign each invoice the tracks of the album referred to by the first With clause
    album_tracks
        AS (
            SELECT
                ift.invoice_id,
                t2.track_id
              FROM invoice_first_track AS ift
             INNER JOIN track AS t1
                ON ift.first_track_id = t1.track_id
             INNER JOIN track AS t2
                ON t1.album_id = t2.album_id
          )

/*
Subquery at FROM clause returns purchase type for each invoice.
It is labeled as 'album' if tracks match those of the corresponding album.
Then group by purchase type and count invoice_id.
*/
SELECT
    purchase_type,
    COUNT(invoice_id) AS number_of_invoices,
    (
     COUNT(invoice_id)/CAST((SELECT COUNT(*) FROM invoice) AS Float)
    )*100 AS percentage
  FROM (
        SELECT
            i.invoice_id,
            CASE
                WHEN (
                      SELECT track_id
                        FROM invoice_tracks
                       WHERE invoice_id = i.invoice_id
                      EXCEPT
                      SELECT track_id
                        FROM album_tracks
                       WHERE invoice_id = i.invoice_id
                    ) IS NULL
                 AND (
                      SELECT track_id
                        FROM album_tracks
                       WHERE invoice_id = i.invoice_id
                      EXCEPT
                      SELECT track_id
                        FROM invoice_tracks
                       WHERE invoice_id = i.invoice_id
                    ) IS NULL
                THEN 'album'
                ELSE 'indivisual'
            END AS purchase_type
          FROM invoice AS i
      )
 GROUP BY purchase_type;

Done.


purchase_type,number_of_invoices,percentage
album,114,18.566775244299674
indivisual,500,81.43322475570032


#### Solution 3

In [34]:
%%sql
/*
For each type of purchase (album or indivisual tracks)
1. Number of invoices
2. Percentage in all purchases
*/

WITH
    --First track of each invoice. Used later as reference to album.
    invoice_first_track
        AS (
            SELECT
                il.invoice_id,
                MIN(il.track_id) AS first_track_id
              FROM invoice_line AS il
             GROUP BY il.invoice_id
          )

/*
Subquery at FROM clause returns purchase type for each invoice.
It is labeled as 'album' if tracks match those of the corresponding album.
Then group by purchase type and count invoice_id.
*/
SELECT
    purchase_type,
    COUNT(invoice_id) AS number_of_invoices,
    (
     COUNT(invoice_id)/CAST((SELECT COUNT(*) FROM invoice) AS Float)
    )*100 AS percentage
  FROM (
        SELECT
            ift.invoice_id,
            CASE
                WHEN (
                      SELECT il.track_id
                        FROM invoice_line AS il
                       WHERE il.invoice_id = ift.invoice_id
                      EXCEPT
                      SELECT t1.track_id
                        FROM track AS t1
                       WHERE t1.album_id = (
                                            --Find the album of the first track of the invoice
                                            SELECT t2.album_id
                                              FROM track AS t2
                                             WHERE t2.track_id = ift.first_track_id
                                           )
                     ) IS NULL
                 AND (
                      SELECT t1.track_id
                        FROM track AS t1
                       WHERE t1.album_id = (
                                            SELECT t2.album_id
                                              FROM track AS t2
                                             WHERE t2.track_id = ift.first_track_id
                                           )
                      EXCEPT
                      SELECT il.track_id
                        FROM invoice_line AS il
                       WHERE il.invoice_id = ift.invoice_id
                     ) IS NULL
                THEN 'album'
                ELSE 'indivisual'
             END AS purchase_type
          FROM invoice_first_track AS ift
        )
 GROUP BY purchase_type;

Done.


purchase_type,number_of_invoices,percentage
album,114,18.566775244299674
indivisual,500,81.43322475570032


### Remarks

Each solution has its own advantage. 

The last solution is probably the most efficient and elegant way. It is nice it's not using many common table expressions, while it might be a little more difficult for readers to grasp the process.

The first solution is adopting a totally different strategy to figure out purchase type and as a result became more lengthy. On the other hand it is probably easier to make sense what each part of the query is doing.

### Conclusion to the store's plan to change purchasing strategy

To conclude, I would suggest NOT to completely get rid of album purchases since nearly 1/5 of invoices consists of album purchases and there is too much chance we would lose customers and revenue.